In [1]:
from src.models import Sequential
from src.layers import *
from src.activation_functions import *
from src.loss_functions import *

import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score, balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


In [2]:
the_image = np.random.random((1, 1, 28, 28))

In [3]:
# weights = np.random.normal(0, stddev, (n_inputs, n_outputs, filter_size, filter_size)).astype(np.float32)

In [4]:
img = np.asarray([[
    [
        [2, 2, 0, 0, 0],
        [2, 0, 1, 0, 1],
        [2, 0, 1, 2, 1],
        [2, 0, 0, 1, 0],
        [0, 2, 1, 0, 1]
    ],
    [
        [2, 1, 1, 2, 0],
        [0, 2, 1, 2, 1],
        [2, 2, 1, 0, 2],
        [2, 2, 2, 2, 0],
        [1, 2, 0, 2, 1]
    ],
    [
        [0, 1, 1, 0, 0],
        [0, 0, 1, 2, 0],
        [2, 0, 1, 1, 2],
        [1, 1, 2, 1, 2],
        [0, 2, 1, 0, 1]
    ],
]])

the_image = np.pad(img, ((0, 0), (0, 0), (1, 1), (1, 1)))

the_image.shape

(1, 3, 7, 7)

In [5]:
weights = np.asarray([
    [
        [
            [1, 0, 0],
            [0, 0, -1],
            [0, 0, -1]
        ],
        [
            [1, 1, 0],
            [-1, 1, -1],
            [1, 1, 1]
        ],
        [
            [0, -1, 1],
            [1, -1, -1],
            [0, -1, 1]
        ]
    ],
    [
        [
            [0, -1, 1],
            [0, 0, -1],
            [1, 0, -1]
        ],
        [
            [1, 1, -1],
            [0, 0, 0],
            [-1, 0, -1]
        ],
        [
            [0, 0, 1],
            [0, 0, -1],
            [0, 1, -1]
        ]
    ],
])

weights.shape

(2, 3, 3, 3)

In [6]:
weights

array([[[[ 1,  0,  0],
         [ 0,  0, -1],
         [ 0,  0, -1]],

        [[ 1,  1,  0],
         [-1,  1, -1],
         [ 1,  1,  1]],

        [[ 0, -1,  1],
         [ 1, -1, -1],
         [ 0, -1,  1]]],


       [[[ 0, -1,  1],
         [ 0,  0, -1],
         [ 1,  0, -1]],

        [[ 1,  1, -1],
         [ 0,  0,  0],
         [-1,  0, -1]],

        [[ 0,  0,  1],
         [ 0,  0, -1],
         [ 0,  1, -1]]]])

In [7]:
input_height = img.shape[2]
input_width = img.shape[3]

new_h = ((input_height - 3 + (2 * 1)) // 2) + 1
new_w = ((input_width - 3 + (2 * 1)) // 2) + 1

output = np.empty((the_image.shape[0], 2, new_h, new_w))
output.shape

(1, 2, 3, 3)

In [8]:
the_image.shape

(1, 3, 7, 7)

In [9]:
bias = np.asarray([1, 0])

In [10]:
# for b in range(output.shape[0]):
for f in range(2):
    for h in range(output.shape[2]):
        for w in range(output.shape[3]):
            output[:, f, h, w] = np.sum(the_image[:, :, h * 2:h * 2+3, w * 2:w * 2+3] * weights[f, :]) + bias[f]


In [11]:
output

array([[[[ 1.,  5.,  2.],
         [ 3.,  4.,  5.],
         [-2.,  1.,  0.]],

        [[-5., -5., -2.],
         [-6., -5.,  3.],
         [-5.,  4.,  2.]]]])

In [12]:
from src.layers import Conv2D

conv = Conv2D(3, 2, 3, 2, 1)

In [17]:
conv.weights = weights.astype(np.float32)

In [18]:
conv.bias = np.asarray([[1, 0]]).astype(np.float32)

In [19]:
out_c = conv.forward(img)
out_c

array([[[[ 1.,  5.,  2.],
         [ 3.,  4.,  5.],
         [-2.,  1.,  0.]],

        [[-5., -5., -2.],
         [-6., -5.,  3.],
         [-5.,  4.,  2.]]]])

In [20]:
conv.backward(out_c, 0.1)

array([[[[4.94065646e-324, 6.95235280e-310, 2.65313252e-321,
                      nan, 6.95235281e-310],
         [0.00000000e+000, 1.95066341e-153, 2.32111980e-152,
          4.55598118e+198, 4.38365256e-019],
         [4.17985905e+222, 6.96747082e+252, 1.06145241e+170,
          4.52745891e+257, 7.94651531e-061],
         [1.29507851e+214, 1.89123196e+219, 3.57725758e-057,
          1.01196552e+200, 1.07014329e+170],
         [4.52745891e+257, 6.96771801e+252, 1.29507851e+214,
          1.89123196e+219, 1.85019266e-070]],

        [[1.29507851e+214, 1.89123196e+219, 7.94651695e-061,
          1.29507851e+214, 1.89123196e+219],
         [2.23674567e-046, 1.22829256e+184, 8.99851346e+130,
          3.18539647e-120, 1.49347735e+214],
         [1.41167386e-095, 2.32111980e-152, 1.61860966e+184,
          2.70038117e-009, 1.35507324e+248],
         [8.18434960e-085, 7.82755902e-120, 1.38497215e+219,
          8.99851360e+130, 3.18539647e-120],
         [3.63521590e+228, 7.82758275e-120, 

In [21]:
conv.weights

array([[[[-1.4000001 ,  0.1       , -0.5       ],
         [-0.8       , -1.7       ,  0.39999998],
         [-1.6       ,  0.7       , -0.5       ]],

        [[-1.4000001 ,  1.1       , -0.5       ],
         [-1.8       , -0.70000005,  0.39999998],
         [-0.6       ,  1.7       ,  1.5       ]],

        [[-2.4       , -0.9       ,  0.5       ],
         [ 0.19999999, -2.7       ,  0.39999998],
         [-1.6       , -0.3       ,  1.5       ]]],


       [[[-1.        , -1.7       ,  2.5       ],
         [-2.8       ,  1.5       ,  1.4000001 ],
         [-0.5       , -0.2       ,  0.5       ]],

        [[ 0.        ,  0.3       ,  0.5       ],
         [-2.8       ,  1.5       ,  2.4       ],
         [-2.5       , -0.2       ,  0.5       ]],

        [[-1.        , -0.7       ,  2.5       ],
         [-2.8       ,  1.5       ,  1.4000001 ],
         [-1.5       ,  0.8       ,  0.5       ]]]], dtype=float32)

In [ ]:
test_error = np.asarray([[
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
]])
print(test_error.shape)

test_error_padded = np.pad(test_error[0], 1)
test_error2 = np.expand_dims(test_error_padded, (0, 1))

In [ ]:
test_error2.shape

In [ ]:
test_error

In [ ]:
weig = np.asarray([[
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
],
[
    [1, 0, -1],
    [2, 0, -2],
    [1, 0, -1]
]])

In [ ]:
weig.shape

In [ ]:
error_conv = np.zeros((1, 1, 6, 5))
error_conv.shape

In [ ]:
for b in range(error_conv.shape[0]):
    for f in range(error_conv.shape[1]):
        for h in range(error_conv.shape[2]):
            for w in range(error_conv.shape[3]):
                error_conv[b, f, h, w] = np.sum(
                    test_error2[b, f, h:h+3, w:w+3] * weig[f]
                )

In [ ]:
error_conv.shape

In [ ]:
error_conv[0]

In [ ]:
import scipy

In [ ]:
scipy.signal.correlate2d(test_error[0][0], weig[0])

In [ ]:
test_error[0][0][1:-1, 1:-1] * error_conv[0][0]

In [ ]:
test_error2 = np.asarray([[
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
]])

test_error = test_error[1:-1, 1:-1]

test_inp = np.asarray([[
    [1, 1, 1, 2, 3],
    [1, 1, 1, 2, 3],
    [1, 1, 1, 2, 3],
    [2, 2, 2, 2, 3],
    [3, 3, 3, 3, 3],
    [4, 4, 4, 4, 4],
], [
    [1, 1, 1, 2, 3],
    [1, 1, 1, 2, 3],
    [1, 1, 1, 2, 3],
    [2, 2, 2, 2, 3],
    [3, 3, 3, 3, 3],
    [4, 4, 4, 4, 4],
]])

test_weig = np.asarray([[
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
],[
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
]])

In [ ]:
test_grad = np.zeros((2, 3, 3))

In [ ]:
# for b in range(test_grad.shape[0]):
for h in range(test_grad.shape[1]):
    for w in range(test_grad.shape[2]):
        test_grad[:, h, w] = np.sum(
            test_inp[:, h:h+test_error.shape[0], w:w+test_error.shape[1]] * test_error
        , axis=(1, 2))

In [ ]:
print(test_grad)

In [ ]:
test_error2 = np.expand_dims(test_error2, 0)

In [ ]:
test_error2.shape

In [ ]:
test_error2

In [ ]:
test_im_err = np.zeros((1, 1, 6, 5))

In [ ]:
test_weig.shape

In [ ]:
# for b in range(test_grad.shape[0]):
for h in range(6):
    for w in range(5):
        test_im_err[:, :, h, w] = np.sum(
            test_error2[:, :, h:h+3, w:w+3] * test_weig)

In [ ]:
test_im_err

In [ ]:
scipy.signal.correlate2d(test_error2[0][0], test_weig[0])